In [ ]:
import requests
from bs4 import BeautifulSoup as BS
session =requests.session()
session.headers
import pandas as pd
import csv
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.107 Safari/537.36'}

In [ ]:
main_url = 'https://www.themoviedb.org/movie?page='

page_url = []
movie_link = []
movie_url = []

for item in range(1,4+1):
    page_url.append(main_url+str(item))
    for page in page_url:
        source_data = requests.get(page,headers=headers).text
        soup_data = BS(source_data,'lxml')
        div = soup_data.find_all('div',class_='card style_1')
        for a in div:
            link = a.find('a')['href']
            x = (common_url+str(link))
            movie_url.append(x)

In [ ]:
common_url = 'https://www.themoviedb.org'

all_movie_data = []

count = 0
crew_list = []
director_list = []



for movie in movie_url:
    source_data1 = requests.get(movie,headers=headers).text
    soup_data1 = BS(source_data1,'lxml')
    all_divs1 = soup_data1.find_all('div',class_='header large border first')
    for item in all_divs1:
        movie_name = item.find('h2')
        m_n =movie_name.find('a').text
        movie_name = item.find('h2')
        m_n =movie_name.find('a').text
        movie_rating = item.find('div',class_='user_score_chart')['data-percent']
        movie_genre = item.find('span',class_='genres').text.strip()
        movie_release_date = item.find('span',class_='release').text.strip()[0:11]
        movie_runtime = item.find('span',class_='runtime').text.strip()   
        movie_link = item.find('h2')
        mu = movie_link.find('a')['href']
        m_url = (common_url+mu)
    for a in item.find_all('p'):
        count += 1
        crew_list.append(a)
        if a.text.strip() == 'Director' or a.text.strip() == 'Director, Writer' or a.text.strip() == 'Director, Screenplay, Story':
            director_position = count-2
            director1 = crew_list[director_position].text
            director_list.append(director1)

        
    data = {
        'Movie' : m_n,
        'Rating' : movie_rating,
        'Genre' : movie_genre,
        'Release Date' : movie_release_date,
        'Runtime' : movie_runtime,
        'Director' : director1,
        'URL' : m_url
        }
    all_movie_data.append(data)
    
            
            
            
    
  

In [ ]:
df = pd.DataFrame(all_movie_data)

In [ ]:
df

In [ ]:
keys = all_movie_data[0].keys()
with open('MovieList.csv','w',newline='') as op_file:
    dict_writer = csv.DictWriter(op_file,keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_movie_data)